In [1]:
from subprocess import call
from glob import glob
from nltk.corpus import stopwords
import os, struct
from tensorflow.core.example import example_pb2
import pyrouge
import shutil
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import *

In [2]:
# cmd = '/root/miniconda2/bin/python run_summarization.py --mode=decode --single_pass=1 --coverage=True --vocab_path=finished_files/vocab --log_root=log --exp_name=myexperiment --data_path=test/temp_file --max_enc_steps=4000'
# generated_path = '/gttp/pointer-generator-tal/log/myexperiment/decode_test_4000maxenc_4beam_35mindec_100maxdec_ckpt-238410/'
# cmd = cmd.split()

vocab_path = '../data/DMQA/finished_files/vocab'
log_root = 'log'
exp_name = 'myexperiment'
data_path= 'test/temp_file'
max_enc_steps = 4000

cmd = ['python',
       'run_summarization.py',
       '--mode=decode',
       '--single_pass=1',
       '--coverage=True',
       '--vocab_path=' + vocab_path,
       '--log_root=' + log_root,
       '--exp_name=' + exp_name,
       '--data_path=' + data_path,
       '--max_enc_steps=' + str(max_enc_steps)]

generated_path = 'log/myexperiment/decode_test_4000maxenc_4beam_35mindec_100maxdec_ckpt-238410/'
max_len = 250
stopwords = set(stopwords.words('english'))
stemmer = PorterStemmer()

In [3]:
def pp(string):
    return ' '.join([stemmer.stem(word.decode('utf8')) for word in string.lower().split() if not word in stopwords])
    
def write_to_file(article, abstract, rel, writer):
    abstract = '<s> '+' '.join(abstract)+' </s>'
    #abstract = abstract.encode('utf8', 'ignore')
    #rel = rel.encode('utf8', 'ignore')
    #article = article.encode('utf8', 'ignore')
    tf_example = example_pb2.Example()
    tf_example.features.feature['abstract'].bytes_list.value.extend([bytes(abstract)])
    tf_example.features.feature['relevancy'].bytes_list.value.extend([bytes(rel)])
    tf_example.features.feature['article'].bytes_list.value.extend([bytes(article)])
    tf_example_str = tf_example.SerializeToString()
    str_len = len(tf_example_str)
    writer.write(struct.pack('q', str_len))
    writer.write(struct.pack('%ds' % str_len, tf_example_str))


def duck_iterator(i):
    duc_folder = 'duc0' + str(i) + 'tokenized/'
    for topic in os.listdir(duc_folder + 'testdata/docs/'):
        topic_folder = duc_folder + 'testdata/docs/' + topic
        if not os.path.isdir(topic_folder):
            continue
        query = ' '.join(open(duc_folder + 'queries/' + topic).readlines())
        model_files = glob(duc_folder + 'models/' + topic[:-1].upper() + '.*')

        topic_texts = [' '.join(open(topic_folder + '/' + file).readlines()).replace('\n', '') for file in
                       os.listdir(topic_folder)]

        abstracts = [' '.join(open(f).readlines()) for f in model_files]
        yield topic_texts, abstracts, query


def count_score(sent, ref):
    ref = pp(ref).split()
    sent = ' '.join(pp(w) for w in sent.lower().split() if not w in stopwords)
    return sum([1. if w in ref else 0. for w in sent.split()])


def get_tfidf_score_func(magic = 1):
    corpus = []
    for i in range(5, 8):
        for topic_texts, _, _ in duck_iterator(i):
            corpus += [pp(t) for t in topic_texts]

    vectorizer = TfidfVectorizer()
    vectorizer.fit_transform(corpus)

    def tfidf_score_func(sent, ref):
        ref = [pp(s) for s in ref.split(' . ')]
        sent = pp(sent)
        v1 = vectorizer.transform([sent])
        v2s = [vectorizer.transform([r]) for r in ref]
        return max([cosine_similarity(v1, v2)[0][0] for v2 in v2s])
    return tfidf_score_func

In [4]:
tfidf_score = get_tfidf_score_func()

In [5]:
class Summary:
    def __init__(self, texts, abstracts, query):
        
        #texts = sorted([(tfidf_score(query, text), text) for text in texts], reverse=True)
        texts = sorted([(tfidf_score(text, ' '.join(abstracts)), text) for text in texts], reverse=True)

        texts = [text[1] for text in texts]
        self.texts = texts
        self.abstracts = abstracts
        self.query = query
        self.summary = []
        self.words = set()
        self.length = 0

    def most_similar(self, sent, text):
        return max([(count_score(s, sent), s) for s in text])[1]

    def add_sum(self, summ):
        text = self.texts.pop(0).split(' . ')
        if len(self.texts) == 0: return True
        found_sents = []
        for sent in summ:
            ms = self.most_similar(sent, text)
            if ms in found_sents:
                continue
            found_sents.append(sent)
            splitted = pp(sent).split()
            length = len(splitted) 
            splitted = set(splitted)       
            if self.length+length > max_len: return True
            if len(splitted - self.words) < int(len(splitted)*0.5): return False
            self.words |= splitted     
            self.summary.append(sent)
            self.length +=length
        return False

    def get(self):
        text = self.texts[0]
        sents = text.split(' . ')
        #score_per_sent = [(count_score(sent, self.query), sent) for sent in sents]
        score_per_sent = [(count_score(sent, ' '.join(self.abstracts)), sent) for sent in sents]

        scores = []
        for score, sent in score_per_sent:
            scores += [score] * (len(sent.split()) + 1)
        scores = str(scores[:-1])
        return text, 'a', scores

def get_summaries(path):
    path = path+'decoded/'
    out = {}
    for file_name in os.listdir(path):
        index = int(file_name.split('_')[0])
        out[index] = open(path+file_name).readlines()
    return out

def rouge_eval(ref_dir, dec_dir):
    """Evaluate the files in ref_dir and dec_dir with pyrouge, returning results_dict"""
    r = pyrouge.Rouge155()
    r.model_filename_pattern = '#ID#_reference_(\d+).txt'
    r.system_filename_pattern = '(\d+)_decoded.txt'
    r.model_dir = ref_dir
    r.system_dir = dec_dir
    return r.convert_and_evaluate()

def evaluate(summaries):
    for path in ['eval/ref', 'eval/dec']:
        if os.path.exists(path): shutil.rmtree(path, True)
        os.mkdir(path)
    for i, summ in enumerate(summaries):
        for j,abs in enumerate(summ.abstracts):
            with open('eval/ref/'+str(i)+'_reference_'+str(j)+'.txt', 'w') as f:
                f.write(abs)
        with open('eval/dec/'+str(i)+'_decoded.txt', 'w') as f:
            f.write(' '.join(summ.summary))
    print rouge_eval('eval/ref/', 'eval/dec/') 

In [6]:
for i in [5, 6, 7]:
    duc_num = i
    done_summaries = []
    summaries = [Summary(texts, abstracts, query) for texts, abstracts, query in duck_iterator(i)]
    while summaries:
        with open('test/temp_file', 'wb') as writer:
            for summ in summaries:
                article, abstract, scores = summ.get()
                write_to_file(article, abstracts, scores, writer)
        call(['rm', '-r', generated_path])     
        call(cmd)
        generated_summaries = get_summaries(generated_path)
        should_remove = []
        for i in range(len(summaries)):
            if summaries[i].add_sum(generated_summaries[i]):
                should_remove.append(i)
        for i in should_remove[::-1]: 
            done_summaries.append(summaries.pop(i))
    evaluate(done_summaries)
    print duc_num

2019-02-14 16:14:03,351 [MainThread  ] [INFO ]  Writing summaries.
2019-02-14 16:14:03,354 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /tmp/tmpik7Xs5/system and model files to /tmp/tmpik7Xs5/model.
2019-02-14 16:14:03,356 [MainThread  ] [INFO ]  Processing files in eval/dec/.
2019-02-14 16:14:03,357 [MainThread  ] [INFO ]  Processing 0_decoded.txt.
2019-02-14 16:14:03,359 [MainThread  ] [INFO ]  Processing 1_decoded.txt.
2019-02-14 16:14:03,361 [MainThread  ] [INFO ]  Processing 2_decoded.txt.
2019-02-14 16:14:03,364 [MainThread  ] [INFO ]  Processing 3_decoded.txt.
2019-02-14 16:14:03,366 [MainThread  ] [INFO ]  Processing 4_decoded.txt.
2019-02-14 16:14:03,368 [MainThread  ] [INFO ]  Processing 5_decoded.txt.
2019-02-14 16:14:03,370 [MainThread  ] [INFO ]  Processing 6_decoded.txt.
2019-02-14 16:14:03,372 [MainThread  ] [INFO ]  Processing 7_decoded.txt.
2019-02-14 16:14:03,374 [MainThread  ] [INFO ]  Processing 8_decoded.txt.
2019-02-14 16:14:03,377 [MainThr

2019-02-14 16:14:03,587 [MainThread  ] [INFO ]  Processing 7_reference_3.txt.
2019-02-14 16:14:03,589 [MainThread  ] [INFO ]  Processing 8_reference_0.txt.
2019-02-14 16:14:03,591 [MainThread  ] [INFO ]  Processing 8_reference_1.txt.
2019-02-14 16:14:03,594 [MainThread  ] [INFO ]  Processing 8_reference_2.txt.
2019-02-14 16:14:03,596 [MainThread  ] [INFO ]  Processing 8_reference_3.txt.
2019-02-14 16:14:03,598 [MainThread  ] [INFO ]  Processing 8_reference_4.txt.
2019-02-14 16:14:03,600 [MainThread  ] [INFO ]  Processing 8_reference_5.txt.
2019-02-14 16:14:03,602 [MainThread  ] [INFO ]  Processing 8_reference_6.txt.
2019-02-14 16:14:03,605 [MainThread  ] [INFO ]  Processing 8_reference_7.txt.
2019-02-14 16:14:03,607 [MainThread  ] [INFO ]  Processing 8_reference_8.txt.
2019-02-14 16:14:03,609 [MainThread  ] [INFO ]  Processing 9_reference_0.txt.
2019-02-14 16:14:03,612 [MainThread  ] [INFO ]  Processing 9_reference_1.txt.
2019-02-14 16:14:03,614 [MainThread  ] [INFO ]  Processing 9_ref

2019-02-14 16:14:03,824 [MainThread  ] [INFO ]  Processing 25_reference_5.txt.
2019-02-14 16:14:03,826 [MainThread  ] [INFO ]  Processing 25_reference_6.txt.
2019-02-14 16:14:03,828 [MainThread  ] [INFO ]  Processing 25_reference_7.txt.
2019-02-14 16:14:03,831 [MainThread  ] [INFO ]  Processing 25_reference_8.txt.
2019-02-14 16:14:03,833 [MainThread  ] [INFO ]  Processing 26_reference_0.txt.
2019-02-14 16:14:03,835 [MainThread  ] [INFO ]  Processing 26_reference_1.txt.
2019-02-14 16:14:03,837 [MainThread  ] [INFO ]  Processing 26_reference_2.txt.
2019-02-14 16:14:03,839 [MainThread  ] [INFO ]  Processing 26_reference_3.txt.
2019-02-14 16:14:03,842 [MainThread  ] [INFO ]  Processing 27_reference_0.txt.
2019-02-14 16:14:03,844 [MainThread  ] [INFO ]  Processing 27_reference_1.txt.
2019-02-14 16:14:03,846 [MainThread  ] [INFO ]  Processing 27_reference_2.txt.
2019-02-14 16:14:03,848 [MainThread  ] [INFO ]  Processing 27_reference_3.txt.
2019-02-14 16:14:03,850 [MainThread  ] [INFO ]  Proc

2019-02-14 16:14:04,055 [MainThread  ] [INFO ]  Processing 43_reference_2.txt.
2019-02-14 16:14:04,057 [MainThread  ] [INFO ]  Processing 43_reference_3.txt.
2019-02-14 16:14:04,060 [MainThread  ] [INFO ]  Processing 43_reference_4.txt.
2019-02-14 16:14:04,062 [MainThread  ] [INFO ]  Processing 43_reference_5.txt.
2019-02-14 16:14:04,064 [MainThread  ] [INFO ]  Processing 43_reference_6.txt.
2019-02-14 16:14:04,066 [MainThread  ] [INFO ]  Processing 43_reference_7.txt.
2019-02-14 16:14:04,069 [MainThread  ] [INFO ]  Processing 43_reference_8.txt.
2019-02-14 16:14:04,071 [MainThread  ] [INFO ]  Processing 44_reference_0.txt.
2019-02-14 16:14:04,073 [MainThread  ] [INFO ]  Processing 44_reference_1.txt.
2019-02-14 16:14:04,075 [MainThread  ] [INFO ]  Processing 44_reference_2.txt.
2019-02-14 16:14:04,077 [MainThread  ] [INFO ]  Processing 44_reference_3.txt.
2019-02-14 16:14:04,079 [MainThread  ] [INFO ]  Processing 45_reference_0.txt.
2019-02-14 16:14:04,082 [MainThread  ] [INFO ]  Proc

---------------------------------------------
1 ROUGE-1 Average_R: 0.42860 (95%-conf.int. 0.41681 - 0.44145)
1 ROUGE-1 Average_P: 0.34603 (95%-conf.int. 0.33550 - 0.35569)
1 ROUGE-1 Average_F: 0.38248 (95%-conf.int. 0.37166 - 0.39257)
---------------------------------------------
1 ROUGE-2 Average_R: 0.08744 (95%-conf.int. 0.07918 - 0.09613)
1 ROUGE-2 Average_P: 0.07056 (95%-conf.int. 0.06397 - 0.07737)
1 ROUGE-2 Average_F: 0.07800 (95%-conf.int. 0.07065 - 0.08549)
---------------------------------------------
1 ROUGE-3 Average_R: 0.02852 (95%-conf.int. 0.02363 - 0.03373)
1 ROUGE-3 Average_P: 0.02296 (95%-conf.int. 0.01909 - 0.02703)
1 ROUGE-3 Average_F: 0.02541 (95%-conf.int. 0.02104 - 0.03013)
---------------------------------------------
1 ROUGE-4 Average_R: 0.01455 (95%-conf.int. 0.01122 - 0.01838)
1 ROUGE-4 Average_P: 0.01166 (95%-conf.int. 0.00905 - 0.01475)
1 ROUGE-4 Average_F: 0.01293 (95%-conf.int. 0.01001 - 0.01637)
---------------------------------------------
1 ROUGE-L Aver

...

---------------------------------------------
1 ROUGE-1 Average_R: 0.47157 (95%-conf.int. 0.45857 - 0.48513)
1 ROUGE-1 Average_P: 0.38165 (95%-conf.int. 0.37184 - 0.39071)
1 ROUGE-1 Average_F: 0.42141 (95%-conf.int. 0.41120 - 0.43172)
---------------------------------------------
1 ROUGE-2 Average_R: 0.11534 (95%-conf.int. 0.10660 - 0.12363)
1 ROUGE-2 Average_P: 0.09338 (95%-conf.int. 0.08638 - 0.10002)
1 ROUGE-2 Average_F: 0.10310 (95%-conf.int. 0.09536 - 0.11046)
---------------------------------------------
1 ROUGE-3 Average_R: 0.04330 (95%-conf.int. 0.03806 - 0.04904)
1 ROUGE-3 Average_P: 0.03503 (95%-conf.int. 0.03079 - 0.03951)
1 ROUGE-3 Average_F: 0.03869 (95%-conf.int. 0.03402 - 0.04378)
---------------------------------------------
1 ROUGE-4 Average_R: 0.02361 (95%-conf.int. 0.01978 - 0.02772)
1 ROUGE-4 Average_P: 0.01906 (95%-conf.int. 0.01599 - 0.02233)
1 ROUGE-4 Average_F: 0.02107 (95%-conf.int. 0.01768 - 0.02465)
---------------------------------------------
1 ROUGE-L Aver

2019-02-14 16:59:55,517 [MainThread  ] [INFO ]  Writing summaries.
2019-02-14 16:59:55,519 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /tmp/tmpp_u8Bo/system and model files to /tmp/tmpp_u8Bo/model.
2019-02-14 16:59:55,521 [MainThread  ] [INFO ]  Processing files in eval/dec/.
2019-02-14 16:59:55,522 [MainThread  ] [INFO ]  Processing 0_decoded.txt.
2019-02-14 16:59:55,524 [MainThread  ] [INFO ]  Processing 1_decoded.txt.
2019-02-14 16:59:55,527 [MainThread  ] [INFO ]  Processing 2_decoded.txt.
2019-02-14 16:59:55,529 [MainThread  ] [INFO ]  Processing 3_decoded.txt.
2019-02-14 16:59:55,531 [MainThread  ] [INFO ]  Processing 4_decoded.txt.
2019-02-14 16:59:55,534 [MainThread  ] [INFO ]  Processing 5_decoded.txt.
2019-02-14 16:59:55,536 [MainThread  ] [INFO ]  Processing 6_decoded.txt.
2019-02-14 16:59:55,538 [MainThread  ] [INFO ]  Processing 7_decoded.txt.
2019-02-14 16:59:55,540 [MainThread  ] [INFO ]  Processing 8_decoded.txt.
2019-02-14 16:59:55,542 [MainThr

2019-02-14 16:59:55,745 [MainThread  ] [INFO ]  Processing 14_reference_0.txt.
2019-02-14 16:59:55,747 [MainThread  ] [INFO ]  Processing 14_reference_1.txt.
2019-02-14 16:59:55,749 [MainThread  ] [INFO ]  Processing 14_reference_2.txt.
2019-02-14 16:59:55,752 [MainThread  ] [INFO ]  Processing 14_reference_3.txt.
2019-02-14 16:59:55,754 [MainThread  ] [INFO ]  Processing 15_reference_0.txt.
2019-02-14 16:59:55,756 [MainThread  ] [INFO ]  Processing 15_reference_1.txt.
2019-02-14 16:59:55,758 [MainThread  ] [INFO ]  Processing 15_reference_2.txt.
2019-02-14 16:59:55,760 [MainThread  ] [INFO ]  Processing 15_reference_3.txt.
2019-02-14 16:59:55,762 [MainThread  ] [INFO ]  Processing 16_reference_0.txt.
2019-02-14 16:59:55,765 [MainThread  ] [INFO ]  Processing 16_reference_1.txt.
2019-02-14 16:59:55,767 [MainThread  ] [INFO ]  Processing 16_reference_2.txt.
2019-02-14 16:59:55,769 [MainThread  ] [INFO ]  Processing 16_reference_3.txt.
2019-02-14 16:59:55,771 [MainThread  ] [INFO ]  Proc

2019-02-14 16:59:55,973 [MainThread  ] [INFO ]  Processing 40_reference_0.txt.
2019-02-14 16:59:55,976 [MainThread  ] [INFO ]  Processing 40_reference_1.txt.
2019-02-14 16:59:55,978 [MainThread  ] [INFO ]  Processing 40_reference_2.txt.
2019-02-14 16:59:55,980 [MainThread  ] [INFO ]  Processing 40_reference_3.txt.
2019-02-14 16:59:55,982 [MainThread  ] [INFO ]  Processing 41_reference_0.txt.
2019-02-14 16:59:55,985 [MainThread  ] [INFO ]  Processing 41_reference_1.txt.
2019-02-14 16:59:55,987 [MainThread  ] [INFO ]  Processing 41_reference_2.txt.
2019-02-14 16:59:55,989 [MainThread  ] [INFO ]  Processing 41_reference_3.txt.
2019-02-14 16:59:55,991 [MainThread  ] [INFO ]  Processing 42_reference_0.txt.
2019-02-14 16:59:55,994 [MainThread  ] [INFO ]  Processing 42_reference_1.txt.
2019-02-14 16:59:55,996 [MainThread  ] [INFO ]  Processing 42_reference_2.txt.
2019-02-14 16:59:55,998 [MainThread  ] [INFO ]  Processing 42_reference_3.txt.
2019-02-14 16:59:56,000 [MainThread  ] [INFO ]  Proc

---------------------------------------------
1 ROUGE-1 Average_R: 0.48136 (95%-conf.int. 0.46885 - 0.49394)
1 ROUGE-1 Average_P: 0.39496 (95%-conf.int. 0.38255 - 0.40695)
1 ROUGE-1 Average_F: 0.43307 (95%-conf.int. 0.42158 - 0.44417)
---------------------------------------------
1 ROUGE-2 Average_R: 0.12895 (95%-conf.int. 0.11954 - 0.13848)
1 ROUGE-2 Average_P: 0.10592 (95%-conf.int. 0.09746 - 0.11393)
1 ROUGE-2 Average_F: 0.11608 (95%-conf.int. 0.10733 - 0.12481)
---------------------------------------------
1 ROUGE-3 Average_R: 0.04880 (95%-conf.int. 0.04308 - 0.05439)
1 ROUGE-3 Average_P: 0.04016 (95%-conf.int. 0.03530 - 0.04482)
1 ROUGE-3 Average_F: 0.04398 (95%-conf.int. 0.03894 - 0.04897)
---------------------------------------------
1 ROUGE-4 Average_R: 0.02454 (95%-conf.int. 0.02073 - 0.02825)
1 ROUGE-4 Average_P: 0.02013 (95%-conf.int. 0.01697 - 0.02321)
1 ROUGE-4 Average_F: 0.02208 (95%-conf.int. 0.01866 - 0.02542)
---------------------------------------------
1 ROUGE-L Aver